In [ ]:
import $ivy.`org.locationtech.geotrellis::geotrellis-raster:3.5.2`
import $ivy.`org.slf4j:slf4j-simple:1.7.30`

In [ ]:
import geotrellis.vector._
import geotrellis.raster._
import geotrellis.raster.geotiff.GeoTiffRasterSource

In [ ]:
val uri = "https://geotrellis-demo.s3.amazonaws.com/cogs/harrisburg-pa/elevation.tif"
val rs = GeoTiffRasterSource(uri)

In [ ]:
val raster = rs.read().get

In [ ]:
raster.tile.histogram


In [ ]:
import geotrellis.proj4.LatLng

// Carlisle, PA
 val geom = Polygon(
   LineString(
     List(
       (-77.25414276123047, 40.168642443446366),
       (-77.1188735961914, 40.168642443446366),
       (-77.1188735961914, 40.24939807617368),
       (-77.25414276123047, 40.24939807617368),
       (-77.25414276123047, 40.168642443446366)
     )
   )
 )
 val geomInRsCrs = geom.reproject(LatLng, rs.crs)

In [ ]:
val raster0 = Raster(raster.tile.band(0), rs.extent)

In [ ]:
import geotrellis.raster.summary.polygonal._
import geotrellis.raster.summary.polygonal.visitors.FastMapHistogramVisitor

val result = raster0.polygonalSummary(geomInRsCrs, FastMapHistogramVisitor)

In [ ]:
val histogram = result.toOption.get
histogram.statistics

In [ ]:
import geotrellis.raster.summary.types.MeanValue
import geotrellis.raster.summary.polygonal.visitors._

class TileMeanVisitor extends TileCombineVisitor[MeanValue] {
  def fromDouble(value: Double): MeanValue = MeanValue(value, 1L)
}

class TileListVisitor extends TileCombineVisitor[List[Double]] {
  def fromDouble(value: Double): List[Double] = List(value)
}

In [ ]:
val mean = raster0.polygonalSummary(geomInRsCrs, new TileMeanVisitor()).toOption.get
mean.sum
mean.count

In [ ]:
import geotrellis.util._

implicit class TileMethodsNotebook(val self: Raster[Tile]) extends MethodExtensions[Raster[Tile]] {
    def polygonalMean(p: Polygon) = self.polygonalSummary(p, new TileMeanVisitor())
}

val mean2 = raster0.polygonalMean(geomInRsCrs)

In [ ]:
mean.sum
mean.count